<a href="https://colab.research.google.com/github/varunkamate/Text_Classification_Using_ML/blob/main/Text_Classification_using_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data_Set_Linkk:==> https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews?select=IMDB+Dataset.csv

Install the tensorflow library

In [1]:
!pip install --upgrade tensorflow


Importing all the required libraaries

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, SpatialDropout1D
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential
from keras.utils import pad_sequences
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import confusion_matrix
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 255)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
#for ignoring the unecessory warnings
import warnings
warnings.filterwarnings("ignore")

Importing the data set

In [5]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("lakshmi25npathi/imdb-dataset-of-50k-movie-reviews")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'imdb-dataset-of-50k-movie-reviews' dataset.
Path to dataset files: /kaggle/input/imdb-dataset-of-50k-movie-reviews


In [6]:
#loading the data set
df=pd.read_csv(f"{path}/IMDB Dataset.csv")

Exploring the data set

In [7]:
df.head()

,review,sentiment
0,"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of v...",positive
1,"A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen-...",positive
2,"I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue is witty and the characters are likable (even the well b...",positive
3,"Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his parents are fighting all the time.<br /><br />This movie is slower than a soap opera... and suddenly, Jake decides to become Rambo and kill the zombie.<br...",negative
4,"Petter Mattei's ""Love in the Time of Money"" is a visually stunning film to watch. Mr. Mattei offers us a vivid portrait about human relations. This is a movie that seems to be telling us what money, power and success do to people in the different situ...",positive


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [9]:
df.shape

(50000, 2)

In [10]:
df.isna().sum()

,0
review,0
sentiment,0


In [11]:
#Balanced data set
df["sentiment"].value_counts()

,count
sentiment,
positive,25000
negative,25000


As the data set contain huge 50k rows of data so to process it we required high computational power. So here we considering only 10000 rows

In [12]:
df=df.iloc[:10000]

In [13]:
df.head()

,review,sentiment
0,"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of v...",positive
1,"A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen-...",positive
2,"I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue is witty and the characters are likable (even the well b...",positive
3,"Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his parents are fighting all the time.<br /><br />This movie is slower than a soap opera... and suddenly, Jake decides to become Rambo and kill the zombie.<br...",negative
4,"Petter Mattei's ""Love in the Time of Money"" is a visually stunning film to watch. Mr. Mattei offers us a vivid portrait about human relations. This is a movie that seems to be telling us what money, power and success do to people in the different situ...",positive


In [14]:
df.shape

(10000, 2)

In [15]:
df['sentiment'].value_counts()

,count
sentiment,
positive,5028
negative,4972


In [16]:
df["review"][10]

'Phil the Alien is one of those quirky films where the humour is based around the oddness of everything rather than actual punchlines.<br /><br />At first it was very odd and pretty funny but as the movie progressed I didn\'t find the jokes or oddness funny anymore.<br /><br />Its a low budget film (thats never a problem in itself), there were some pretty interesting characters, but eventually I just lost interest.<br /><br />I imagine this film would appeal to a stoner who is currently partaking.<br /><br />For something similar but better try "Brother from another planet"'

In [17]:
df.isna().sum()

,0
review,0
sentiment,0


Here it contain 17 duplicated rows so we droping the duplicated rows from the data set

In [18]:
df.duplicated().sum()

np.int64(17)

In [19]:
df=df.drop_duplicates()

In [20]:
df.shape

(9983, 2)

#Basic data preprocessing

Removing HTML tags

Removing Stop_words

Converting to lower case

In [21]:
import re
def remove_html_tags(text):
  cleaned_text=re.sub(r'<.*?>', '', text)
  return cleaned_text

In [22]:
df["review"]=df["review"].apply(remove_html_tags)

In [23]:
df["review"][10]

'Phil the Alien is one of those quirky films where the humour is based around the oddness of everything rather than actual punchlines.At first it was very odd and pretty funny but as the movie progressed I didn\'t find the jokes or oddness funny anymore.Its a low budget film (thats never a problem in itself), there were some pretty interesting characters, but eventually I just lost interest.I imagine this film would appeal to a stoner who is currently partaking.For something similar but better try "Brother from another planet"'

Converting to lower case

In [24]:
df["review"]=df["review"].apply(lambda x:x.lower())

In [25]:
df["review"][10]

'phil the alien is one of those quirky films where the humour is based around the oddness of everything rather than actual punchlines.at first it was very odd and pretty funny but as the movie progressed i didn\'t find the jokes or oddness funny anymore.its a low budget film (thats never a problem in itself), there were some pretty interesting characters, but eventually i just lost interest.i imagine this film would appeal to a stoner who is currently partaking.for something similar but better try "brother from another planet"'

Removing stop words

In [26]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
nltk.download('stopwords')

import nltk
nltk.download('punkt_tab')

def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text)
    filtered = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [27]:

df["review"]=df["review"].apply(remove_stopwords)

In [28]:
df["review"][10]

"phil alien one quirky films humour based around oddness everything rather actual punchlines.at first odd pretty funny movie progressed n't find jokes oddness funny anymore.its low budget film ( thats never problem ) , pretty interesting characters , eventually lost interest.i imagine film would appeal stoner currently partaking.for something similar better try `` brother another planet ''"

Url removal

In [29]:
def remove_url(text):
  clean_text = re.sub(r'http\S+|www\.\S+', '', text)
  return clean_text

In [30]:
df["review"]=df["review"].apply(remove_url)

In [31]:
df["review"][10]

"phil alien one quirky films humour based around oddness everything rather actual punchlines.at first odd pretty funny movie progressed n't find jokes oddness funny anymore.its low budget film ( thats never problem ) , pretty interesting characters , eventually lost interest.i imagine film would appeal stoner currently partaking.for something similar better try `` brother another planet ''"

spliting the data into input and output parts

In [32]:
#for input
x=df["review"]

#for output
y=df["sentiment"]

In [33]:
x.head()

,review
0,"one reviewers mentioned watching 1 oz episode 'll hooked . right , exactly happened me.the first thing struck oz brutality unflinching scenes violence , set right word go . trust , show faint hearted timid . show pulls punches regards drugs , sex viol..."
1,"wonderful little production . filming technique unassuming- old-time-bbc fashion gives comforting , sometimes discomforting , sense realism entire piece . actors extremely well chosen- michael sheen `` got polari '' voices pat ! truly see seamless edi..."
2,"thought wonderful way spend time hot summer weekend , sitting air conditioned theater watching light-hearted comedy . plot simplistic , dialogue witty characters likable ( even well bread suspected serial killer ) . may disappointed realize match poin..."
3,"basically 's family little boy ( jake ) thinks 's zombie closet & parents fighting time.this movie slower soap opera ... suddenly , jake decides become rambo kill zombie.ok , first 're going make film must decide thriller drama ! drama movie watchable..."
4,"petter mattei 's `` love time money '' visually stunning film watch . mr. mattei offers us vivid portrait human relations . movie seems telling us money , power success people different situations encounter . variation arthur schnitzler 's play theme ..."


In [34]:
y.head()

,sentiment
0,positive
1,positive
2,positive
3,negative
4,positive


Encoding for sentiment column

In [35]:
from sklearn.preprocessing import LabelEncoder

In [36]:
encoder=LabelEncoder()

y=encoder.fit_transform(y)

In [37]:
y

array([1, 1, 1, ..., 0, 0, 1])

Calling the sklearn library for train and test data spliting

In [38]:
from sklearn.model_selection import train_test_split

In [39]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [40]:
x_train.shape

(7986,)

#BOW

In [41]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer()

In [42]:
x_train_bow = cv.fit_transform(x_train).toarray()
x_test_bow = cv.transform(x_test).toarray()

In [43]:
x_train_bow

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [44]:
from sklearn.naive_bayes import GaussianNB
gb_model=GaussianNB()

In [45]:
gb_model.fit(x_train_bow,y_train)

GaussianNB()

In [46]:
y_pred=gb_model.predict(x_test_bow)

In [47]:
from sklearn.metrics import accuracy_score

In [48]:
print(accuracy_score(y_test,y_pred))

0.629444166249374


In [49]:
from sklearn.ensemble import RandomForestClassifier
rn_bow=RandomForestClassifier()
rn_bow.fit(x_train_bow,y_train)

RandomForestClassifier()

In [50]:
y_pred=rn_bow.predict(x_test_bow)
print(accuracy_score(y_test,y_pred))

0.8602904356534802


#N_GRAMS

In [51]:
cv = CountVectorizer(ngram_range=(1,2),max_features=5000)

x_train_bow = cv.fit_transform(x_train).toarray()
x_test_bow = cv.transform(x_test).toarray()

rn_ngram=RandomForestClassifier()
rn_ngram.fit(x_train_bow,y_train)

RandomForestClassifier()

In [52]:
y_pred=rn_ngram.predict(x_test_bow)
print(accuracy_score(y_test,y_pred))

0.8402603905858789


#TFIDF

In [53]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer()

In [54]:
x_train_bow = tfidf.fit_transform(x_train).toarray()
x_test_bow = tfidf.transform(x_test).toarray()

rn_tfidf=RandomForestClassifier()
rn_tfidf.fit(x_train_bow,y_train)

RandomForestClassifier()

In [55]:
y_pred=rn_tfidf.predict(x_test_bow)
print(accuracy_score(y_test,y_pred))

0.8512769153730596


By considering above all the model accuracy randomforest with TFIDF achieved more accuracy so consider this model  is best for this project

In [56]:
import joblib

joblib.dump(rn_tfidf,"model.pkl")

['model.pkl']

In [57]:
joblib.load("model.pkl")

RandomForestClassifier()